<a href="https://colab.research.google.com/github/Fercho1983/conciliaciones/blob/main/conciliaciones_bancarias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from datetime import datetime
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter
import pandas as pd
from google.colab import files
import re

# Instalar openpyxl si es necesario
!pip install -q openpyxl

# Subida de archivos
print("🔼 Sube el archivo de BANCOS")
uploaded_bancos = files.upload()
print("🔼 Sube el archivo de LIBROS")
uploaded_libros = files.upload()
file_bancos = list(uploaded_bancos.keys())[0]
file_libros = list(uploaded_libros.keys())[0]

# Detectar encabezado
def detectar_encabezado(file):
    temp_df = pd.read_excel(file, header=None)
    for i in range(min(20, len(temp_df))):
        fila = temp_df.iloc[i].astype(str).str.lower().str.strip()
        if any("fecha" in c for c in fila):
            return i
    return 0

skip_bancos = detectar_encabezado(file_bancos)
skip_libros = detectar_encabezado(file_libros)

# Leer archivos
df_bancos = pd.read_excel(file_bancos, skiprows=skip_bancos)
df_libros = pd.read_excel(file_libros, skiprows=skip_libros)
df_bancos.columns = df_bancos.columns.str.strip().str.lower()
df_libros.columns = df_libros.columns.str.strip().str.lower()

# Extraer nombre de la compañía desde archivo de libros (buscando columnas "cliente", "empresa" o "compañia")
compania_cols = [col for col in df_libros.columns if re.search(r"cliente|empresa|compañia", col, re.IGNORECASE)]
if compania_cols:
    primera_col = compania_cols[0]
    valor_col = str(df_libros[primera_col].dropna().iloc[0])
    nombre_compania = valor_col.strip() if valor_col else "Nombre de la Compañía"
else:
    nombre_compania = "Nombre de la Compañía"

# Validación de columnas
obligatorias_bancos = ["fecha", "débito", "crédito"]
obligatorias_libros = ["fecha", "depósito/reg.", "pagado/cr."]
for col in obligatorias_bancos:
    if not any(col in c for c in df_bancos.columns):
        raise ValueError(f"Falta columna '{col}' en archivo de bancos")
for col in obligatorias_libros:
    if not any(col in c for c in df_libros.columns):
        raise ValueError(f"Falta columna '{col}' en archivo de libros")

# Renombrar columnas
df_bancos = df_bancos.rename(columns={
    "fecha": "fecha_bancos", "descripción": "descripcion_banco", "débito": "debito", "crédito": "credito"
})
df_libros = df_libros.rename(columns={
    "fecha": "fecha_libros", "transf.#": "transf_num", "j#": "je",
    "referencia": "referencia_libros", "depósito/reg.": "deposito", "pagado/cr.": "pagado"
})

# Normalizar fechas
t_meses = {"ene": "jan", "feb": "feb", "mar": "mar", "abr": "apr", "may": "may", "jun": "jun",
           "jul": "jul", "ago": "aug", "set": "sep", "oct": "oct", "nov": "nov", "dic": "dec"}
def normalizar_fecha(fecha):
    if pd.isna(fecha):
        return pd.NaT
    texto = str(fecha).lower()
    for esp, eng in t_meses.items():
        texto = re.sub(rf"\b{esp}\b", eng, texto)
    return pd.to_datetime(texto, dayfirst=True, errors="coerce")

df_bancos["fecha_bancos"] = df_bancos["fecha_bancos"].apply(normalizar_fecha)
df_libros["fecha_libros"] = df_libros["fecha_libros"].apply(normalizar_fecha)
df_bancos = df_bancos[df_bancos["fecha_bancos"].notna()]
df_libros = df_libros[df_libros["fecha_libros"].notna()]

# Convertir montos
def convertir_monto(x):
    if pd.isna(x):
        return 0
    x = str(x).replace("(", "-").replace(")", "")
    x = re.sub(r"[^\d,.-]", "", x)
    if x.count(',') > 0 and x.count('.') > 0:
        if x.find('.') < x.find(','):
            x = x.replace('.', '').replace(',', '.')
        else:
            x = x.replace(',', '')
    elif x.count(',') > 0:
        x = x.replace('.', '').replace(',', '.')
    try:
        return float(x)
    except:
        return 0

df_bancos["monto_banco"] = df_bancos.get("credito", 0).apply(convertir_monto) - df_bancos.get("debito", 0).apply(convertir_monto)
df_libros["monto_libros"] = df_libros.get("deposito", 0).apply(convertir_monto) - df_libros.get("pagado", 0).apply(convertir_monto)

# Conciliación automática
df_bancos["match"] = False
df_libros["match"] = False
conciliados = []

for _, banco in df_bancos.iterrows():
    fecha_banco = banco["fecha_bancos"]
    monto_banco = round(banco["monto_banco"], 2)
    candidatos = df_libros[
        (~df_libros["match"]) &
        (df_libros["monto_libros"].round(2) == monto_banco) &
        (df_libros["fecha_libros"].between(fecha_banco - pd.Timedelta(days=2), fecha_banco + pd.Timedelta(days=2)))
    ]
    if not candidatos.empty:
        libro = candidatos.iloc[0]
        df_libros.at[libro.name, "match"] = True
        df_bancos.at[banco.name, "match"] = True
        conciliados.append({**banco, **libro, "estado": "Conciliado ±2 días"})

solo_banco = df_bancos[~df_bancos["match"]].copy()
solo_banco["estado"] = "Solo en banco"
solo_libros = df_libros[~df_libros["match"]].copy()
solo_libros["estado"] = "Solo en libros"

columnas = ["fecha_bancos", "descripcion_banco", "monto_banco", "fecha_libros", "referencia_libros", "monto_libros", "estado"]
def asegurar_columnas(df):
    for col in columnas:
        if col not in df.columns:
            df[col] = ""
    return df[columnas]

df_final = pd.concat([
    asegurar_columnas(pd.DataFrame(conciliados)),
    asegurar_columnas(solo_banco),
    asegurar_columnas(solo_libros)
], ignore_index=True)

# Orden cronológico y diferencia
df_final["fecha_orden"] = df_final["fecha_bancos"].combine_first(df_final["fecha_libros"])
df_final["fecha_orden"] = pd.to_datetime(df_final["fecha_orden"], errors="coerce")
df_final = df_final.sort_values("fecha_orden").drop(columns="fecha_orden")
df_final["monto_banco"] = pd.to_numeric(df_final["monto_banco"], errors="coerce").fillna(0)
df_final["monto_libros"] = pd.to_numeric(df_final["monto_libros"], errors="coerce").fillna(0)
df_final["diferencia"] = df_final["monto_banco"] - df_final["monto_libros"]

# Crear Excel y hoja "Conciliación"
wb = Workbook()
ws = wb.active
ws.title = "Conciliación"
ws.sheet_view.showGridLines = False
ws.freeze_panes = "A2"

encabezado_formateado = {
    "fecha_bancos": "Fecha Bancos", "descripcion_banco": "Descripción Banco", "monto_banco": "Monto Banco",
    "fecha_libros": "Fecha Libros", "referencia_libros": "Referencia Libros", "monto_libros": "Monto Libros",
    "estado": "Estado", "diferencia": "Diferencia"
}
ws.append([encabezado_formateado.get(col, col.title()) for col in df_final.columns])

# Formato del encabezado de la hoja "Conciliación"
for cell in ws[1]:
    cell.font = Font(bold=True, color="FFFFFF")
    cell.alignment = Alignment(horizontal="center", vertical="center")
    cell.fill = PatternFill("solid", fgColor="305496")

fill_gris_claro = PatternFill("solid", fgColor="F2F2F2")
from openpyxl.utils.dataframe import dataframe_to_rows
for i, row in enumerate(dataframe_to_rows(df_final, index=False, header=False), start=2):
    ws.append(row)
    if i % 2 == 1:
        for cell in ws[i]:
            cell.fill = fill_gris_claro

for fila in ws.iter_rows(min_row=2, max_row=ws.max_row):
    for i, celda in enumerate(fila):
        if i in [0, 3]:
            celda.number_format = 'DD-MM-YYYY'
            celda.alignment = Alignment(horizontal="center")
        elif i in [2, 5, 6, 7]:
            celda.number_format = '#,##0.00'
            celda.alignment = Alignment(horizontal="right")
        else:
            celda.alignment = Alignment(horizontal="left")
        if i in [0, 1, 2]:
            celda.font = Font(color="0000FF")

totales = [""] * len(df_final.columns)
totales[df_final.columns.get_loc("descripcion_banco")] = "Total bancos"
totales[df_final.columns.get_loc("monto_banco")] = df_final["monto_banco"].sum()
totales[df_final.columns.get_loc("referencia_libros")] = "Total libros"
totales[df_final.columns.get_loc("monto_libros")] = df_final["monto_libros"].sum()
totales[df_final.columns.get_loc("estado")] = "Total diferencia"
totales[df_final.columns.get_loc("diferencia")] = df_final["diferencia"].sum()
ws.append(totales)
for i, cell in enumerate(ws[ws.max_row], start=1):
    if cell.value:
        cell.font = Font(bold=True)
        if i-1 in [2, 5, 6, 7]:
            cell.number_format = '#,##0.00'

for col in ws.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))
    ws.column_dimensions[column].width = max(12, min(50, max_length + 2))

# Función agregar_hoja_caratula() con definición completa
def agregar_hoja_caratula(wb, compania, banco, cuenta_corriente, cuenta_cliente, fecha_conciliacion, saldo_bancos, saldo_libros):
    ws = wb.create_sheet(title="Carátula", index=0)
    ws.sheet_view.zoomScale = 80
    ws.sheet_view.showGridLines = False

    gris_fondo = "F2F2F2"
    borde_negro = Border(bottom=Side(style="thin", color="000000"))
    borde_gris = Border(bottom=Side(style="thin", color="AAAAAA"))

    # Función para insertar celdas formateadas
    def celda(fila, col, valor, negrita=False, merge=1, tam=12, align="left", bg=None):
        if merge > 1:
            ws.merge_cells(start_row=fila, start_column=col, end_row=fila, end_column=col + merge - 1)
        c = ws.cell(row=fila, column=col)
        c.value = valor
        c.font = Font(bold=negrita, size=tam, color="000000")
        c.alignment = Alignment(horizontal=align)
        if bg:
            c.fill = PatternFill("solid", fgColor=bg)

    # Encabezado centrado
    celda(1, 2, "Conciliación bancaria saldos ajustados", True, 4, 16, "center")
    celda(3, 2, compania, True, 4, 14, "center")
    celda(4, 2, banco, False, 4, 12, "center")
    celda(5, 2, f"Cuenta corriente: {cuenta_corriente}", False, 4, 12, "center")
    celda(6, 2, f"Cuenta cliente: {cuenta_cliente}", False, 4, 12, "center")
    celda(7, 2, fecha_conciliacion.strftime("%d de %B de %Y").capitalize(), False, 4, 12, "center")

    # Saldo bancos
    celda(9, 2, "Saldo en bancos", True, 3)
    celda(9, 5, f"₡{saldo_bancos:,.2f}", True, align="right")
    for col in range(2, 6):
        ws.cell(row=9, column=col).border = borde_negro

    celda(11, 2, "Menos:", True)
    celda(12, 2, "Cheques pendientes de cambio")
    celda(13, 2, "Notas de débito")
    celda(14, 2, "Saldos no aplicados")

    celda(16, 2, "Más:", True)
    celda(17, 2, "Cheques en tránsito")
    celda(18, 2, "Notas de crédito")

    celda(20, 2, "Saldo ajustado bancos", True, 3, bg=gris_fondo)
    celda(20, 5, f"₡{saldo_bancos:,.2f}", True, align="right", bg=gris_fondo)
    for col in range(2, 6):
        ws.cell(row=20, column=col).border = borde_negro

    celda(23, 2, "Disponibilidad bancaria al cierre", True, 3, bg=gris_fondo)
    celda(23, 5, f"₡{saldo_bancos:,.2f}", True, align="right", bg=gris_fondo)
    ws.cell(row=23, column=2).border = borde_negro

    # Saldo libros
    celda(26, 2, "Saldo según contabilidad", True, 3)
    celda(26, 5, f"₡{saldo_libros:,.2f}", True, align="right")
    for col in range(2, 6):
        ws.cell(row=26, column=col).border = borde_negro

    celda(28, 2, "Menos:", True)
    celda(29, 2, "Notas de débito")
    celda(30, 2, "Saldos no aplicados")

    celda(32, 2, "Más:", True)
    celda(33, 2, "Cheques en tránsito")
    celda(34, 2, "Notas de crédito")

    celda(36, 2, "Saldo ajustado libros", True, 3, bg=gris_fondo)
    celda(36, 5, f"₡{saldo_libros:,.2f}", True, align="right", bg=gris_fondo)
    for col in range(2, 6):
        ws.cell(row=36, column=col).border = borde_negro

    # Firmas con línea y texto
    def firma_con_borde_y_etiqueta(fila, texto):
        celda(fila, 2, texto, negrita=True)
        for col in range(2, 6):
            ws.cell(row=fila + 1, column=col).border = borde_negro
        celda(fila + 2, 2, "Nombre y firma", tam=10, align="center", merge=4)

    firma_con_borde_y_etiqueta(40, "Elaborado por:")
    firma_con_borde_y_etiqueta(44, "Revisado por:")
    firma_con_borde_y_etiqueta(48, "Aprobado por:")

    # Ajustar anchos automáticamente
    for col in ws.columns:
        max_len = max(len(str(cell.value)) if cell.value else 0 for cell in col)
        ancho = max(12, min(50, max_len + 2))
        ws.column_dimensions[get_column_letter(col[0].column)].width = ancho



# Leer el archivo sin encabezado para extraer datos de la carátula
df_encabezado_bancos = pd.read_excel(file_bancos, header=None)

# Extraer campos desde posiciones conocidas (líneas 5 a 9, columna 6 = índice 6)
try:
    nombre_compania = str(df_encabezado_bancos.iloc[4, 6]).strip() if pd.notna(df_encabezado_bancos.iloc[4, 6]) else "Nombre de la Compañía"
    cuenta_cliente = str(df_encabezado_bancos.iloc[5, 6]).strip() if pd.notna(df_encabezado_bancos.iloc[5, 6]) else ""
    tipo_cuenta = str(df_encabezado_bancos.iloc[6, 6]).strip() if pd.notna(df_encabezado_bancos.iloc[6, 6]) else ""
    nombre_banco = str(df_encabezado_bancos.iloc[7, 6]).strip() if pd.notna(df_encabezado_bancos.iloc[7, 6]) else "Banco"
    moneda = str(df_encabezado_bancos.iloc[8, 6]).strip() if pd.notna(df_encabezado_bancos.iloc[8, 6]) else ""
except:
    nombre_compania = "Nombre de la Compañía"
    cuenta_cliente = ""
    tipo_cuenta = ""
    nombre_banco = "Banco"
    moneda = ""

# Asignar cuenta corriente si la detectamos como parte del número IBAN
cuenta_corriente = cuenta_cliente[-9:] if len(cuenta_cliente) >= 9 else ""




agregar_hoja_caratula(
    wb,
    compania=nombre_compania,
    banco=nombre_banco,
    cuenta_corriente=cuenta_corriente,
    cuenta_cliente=cuenta_cliente,
    fecha_conciliacion=datetime(2025, 4, 30),
    saldo_bancos=round(df_final["monto_banco"].sum(), 2),
    saldo_libros=round(df_final["monto_libros"].sum(), 2)
)

# Exportar archivo
archivo = "conciliacion_final.xlsx"
wb.save(archivo)
files.download(archivo)




🔼 Sube el archivo de BANCOS


Saving banco porter colones.xlsx to banco porter colones (18).xlsx
🔼 Sube el archivo de LIBROS


Saving porter libros colones.xlsx to porter libros colones (18).xlsx


<ipython-input-22-246804305>:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_libros["monto_libros"] = df_libros.get("deposito", 0).apply(convertir_monto) - df_libros.get("pagado", 0).apply(convertir_monto)
<ipython-input-22-246804305>:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_libros["match"] = False


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>